In [2]:
# I want to predict the individual teams total points and add them together so build dataframes with teams offensive stats and opponents defensive stats

# Retrieve data, clean data and store in folder to train model on

In [3]:
# Imports

import nflreadpy as nfl
import pandas as pd

## Retrieve data

In [7]:
schedule = nfl.load_schedules(seasons=2025)
schedule = schedule.to_pandas()

team_stats = nfl.load_team_stats(seasons=2025)
team_stats = team_stats.to_pandas()

In [12]:
team_stats

,week,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,...,def_safeties,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_att,fg_pct,pat_pct
0,1,ARI,NO,21,29,163,2,0,5,-33,...,0,9,54,3,30,3,73,3,0.666667,1.0
1,2,ARI,CAR,17,25,220,1,1,1,-9,...,0,12,96,1,29,4,87,2,1.000000,1.0
2,3,ARI,SF,22,35,159,1,0,1,-5,...,1,5,30,1,8,4,115,3,0.666667,1.0
3,4,ARI,SEA,27,41,200,2,2,6,-36,...,0,7,46,1,15,5,126,2,1.000000,1.0
4,5,ARI,TEN,23,32,220,0,0,3,-28,...,0,8,47,2,16,4,90,0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2,WAS,GB,24,42,200,2,0,4,-21,...,0,5,30,3,50,3,103,3,0.333333,1.0
184,3,WAS,LV,15,21,207,1,0,1,-8,...,0,7,54,5,127,3,118,3,0.666667,1.0
185,4,WAS,ATL,16,27,156,2,1,2,-9,...,0,6,50,2,0,6,199,4,1.000000,1.0
186,5,WAS,LAC,15,26,231,1,0,1,-5,...,0,7,48,0,0,3,92,2,1.000000,1.0


In [11]:
# Filter out data I don't want

cols_to_drop = ['season', 
 'season_type',
 'sack_fumbles',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_2pt_conversions',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_2pt_conversions',
 'special_teams_tds',
 'misc_yards',
 'fumble_recovery_own',
 'fumble_recovery_yards_own',
 'fumble_recovery_opp',
 'fumble_recovery_yards_opp',
 'fumble_recovery_tds',
 'timeouts',
 'fg_made',
 'fg_missed',
 'fg_blocked',
 'fg_long',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_made_list',
 'fg_missed_list',
 'fg_blocked_list',
 'fg_made_distance',
 'fg_missed_distance',
 'fg_blocked_distance', 
 'pat_made',
 'pat_att',
 'pat_missed',
 'pat_blocked',
 'gwfg_made',
 'gwfg_att',
 'gwfg_missed',
 'gwfg_blocked',
 'gwfg_distance']

team_stats = team_stats.drop(columns=cols_to_drop)

In [9]:
# for each week group by team sum/avg all previous weeks 
team_stats = team_stats.sort_values(["team", "week"]).reset_index(drop=True)

In [14]:
# Cumulative sums up to certain week

sum_cols = ['completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'passing_interceptions',
 'sacks_suffered',
 'sack_yards_lost',
 'sack_fumbles_lost',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'receiving_fumbles_lost',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_fumbles_forced',
 'def_sacks',
 'def_sack_yards',
 'def_qb_hits',
 'def_interceptions',
 'def_interception_yards',
 'def_pass_defended',
 'def_tds',
 'def_fumbles',
 'def_safeties',
 'penalties',
 'penalty_yards',
 'punt_returns',
 'punt_return_yards',
 'kickoff_returns',
 'kickoff_return_yards',
 'fg_att']

team_stats[[f"sum_{c}" for c in sum_cols]] = team_stats.groupby("team")[sum_cols].cumsum()

In [15]:
# Cumulative averages up to a certain week

avg_cols = ['passing_epa',
 'passing_cpoe',
 'rushing_epa',
 'receiving_epa',
 'fg_pct',
 'pat_pct']

team_stats[[f"avg_{c}" for c in avg_cols]] = (
    team_stats.groupby("team")[avg_cols]
      .expanding()
      .mean()
      .reset_index(level=0, drop=True)
)


In [16]:
team_stats

,week,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,...,sum_punt_return_yards,sum_kickoff_returns,sum_kickoff_return_yards,sum_fg_att,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,avg_receiving_epa,avg_fg_pct,avg_pat_pct
0,1,ARI,NO,21,29,163,2,0,5,-33,...,30,3,73,3,1.524799,-1.733811,1.158540,8.777653,0.666667,1.0
1,2,ARI,CAR,17,25,220,1,1,1,-9,...,59,7,160,5,3.832284,-0.485040,0.494152,8.592124,0.833333,1.0
2,3,ARI,SF,22,35,159,1,0,1,-5,...,67,11,275,8,2.348651,-1.198549,0.300113,6.395634,0.777778,1.0
3,4,ARI,SEA,27,41,200,2,2,6,-36,...,82,16,401,10,0.935389,-2.138376,-0.249674,6.292581,0.833333,1.0
4,5,ARI,TEN,23,32,220,0,0,3,-28,...,98,20,491,10,0.988292,-0.585992,-1.390317,6.789066,0.833333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2,WAS,GB,24,42,200,2,0,4,-21,...,50,6,189,3,-3.896936,-4.243721,2.609171,3.567202,0.333333,1.0
184,3,WAS,LV,15,21,207,1,0,1,-8,...,177,9,307,6,0.996192,2.028439,3.076621,7.561061,0.500000,1.0
185,4,WAS,ATL,16,27,156,2,1,2,-9,...,177,15,506,10,0.735543,2.307131,3.503518,7.108524,0.666667,1.0
186,5,WAS,LAC,15,26,231,1,0,1,-5,...,177,18,598,12,3.389712,1.695046,2.802895,8.699414,0.750000,1.0
